In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyp
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

C:\Users\IMOE001\AppData\Local\Temp\ipykernel_6440\4016906179.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
import json

# take legitimates after pre-processing
with open("../Input/preprocess_legitimate_36400.json", "r") as f:
    legit_dict_lst = json.loads(f.read())[:10] # just for check

# take legitimates after pre-processing
with open("../Input/preprocess_phishing_37175.json", "r") as f:
    phish_dict_lst = json.loads(f.read())[:10] # just for check


print(legit_dict_lst)
print(phish_dict_lst)

[{'url': 'http://www.conceptdraw.com/How-To-Guide/Local-Area-Network', 'domain': 'conceptdraw', 'registered_domain': 'conceptdraw.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 0, 'protocol': 'http', 'path': '/How-To-Guide/Local-Area-Network', 'words_raw': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'www'], 'brand_words': [], 'keyword_words': [], 'random_words': ['conceptdraw', 'www'], 'word_list': ['how', 'to', 'guide', 'local', 'area', 'network'], 'word_composed': [], 'word_malicious': []}, {'url': 'https://www.edrawsoft.com/Local-Area-Network.php', 'domain': 'edrawsoft', 'registered_domain': 'edrawsoft.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 1, 'protocol': 'https', 'path': '/Local-Area-Network.php', 'words_raw': ['edrawsoft', 'local', 'area', 'network', 'php', 'www'], 'brand_words': [], 'keyword_words': [], 'random_words': ['php', 'www'], 'word_list': ['local', 'area', 'network', 'edrawsoft'], 'word_composed': 

In [77]:
column_names = ["id", "url", "class", "word_raw_count", "word_brand_count", "avg_word_raw_len", "longest_raw_len", "shortest_raw_len", "word_composed_count", "avg_word_composed_len", "word_keyword_count", "word_malicious_count", "word_random_count", "word_list_count", "digit_count", "subdomain_len", "subdomain_count", "url_len", "www", "com", "special_character_count", "http", "https", "known_TLD"]
len(column_names)

24

In [78]:
def avg_word_len(lst):
    if not lst: return 0
    lens = [len(word) for word in lst]
    return sum(lens)//len(lens)

def max_word_len(lst):
    if not lst: return 0
    return max([len(word) for word in lst])

def min_word_len(lst):
    if not lst: return 0
    return min([len(word) for word in lst])

# load json to df with feature exstraction
def get_urls(list_of_dicts):
    return [d["url"] for d in list_of_dicts]

def get_ids(list_of_dicts):
    return [d["id"] for d in list_of_dicts]

def get_classes(list_of_dicts):
    return [d["class"] for d in list_of_dicts]



def get_word_raw_counts(list_of_dicts):
    return [len(d["words_raw"]) for d in list_of_dicts]

def get_word_list_counts(list_of_dicts):
    return [len(d["word_list"]) for d in list_of_dicts]

def get_word_brand_counts(list_of_dicts):
    return [len(d["brand_words"]) for d in list_of_dicts]

def get_word_keyword_counts(list_of_dicts):
    return [len(d["keyword_words"]) for d in list_of_dicts]

def get_word_random_counts(list_of_dicts):
    return [len(d["random_words"]) for d in list_of_dicts]

def get_word_composed_counts(list_of_dicts):
    return [len(d["word_composed"]) for d in list_of_dicts]

def get_word_malisious_counts(list_of_dicts):
    return [len(d["word_malicious"]) for d in list_of_dicts]



def get_avg_word_raw_len(list_of_dicts):
    return [avg_word_len(d["words_raw"]) for d in list_of_dicts]

def get_avg_word_composed_len(list_of_dicts):
    return [avg_word_len(d["word_composed"]) for d in list_of_dicts]



def get_longest_raw_lens(list_of_dicts):
    return [max_word_len(d["words_raw"]) for d in list_of_dicts]

def get_shortest_raw_lens(list_of_dicts):
    return [min_word_len(d["words_raw"]) for d in list_of_dicts]


def get_digit_counts(list_of_dicts):
    ret = []
    for d in list_of_dicts:
        sum = 0
        for i in range(10):
            sum += d["url"].count(str(i))
        ret.append(sum)
    
    return ret


def get_subdomain_lens(list_of_dicts):
    return [len(d["subdomain"]) for d in list_of_dicts]

def get_subdomain_counts(list_of_dicts):
    return [len(d["subdomain"].split('.')) if d["subdomain"] else 0 for d in list_of_dicts]

def get_url_lens(list_of_dicts):
    return [len(d["url"]) for d in list_of_dicts]

def get_word_is_in_urls(list_of_dicts, word):
    return [word in d["url"] for d in list_of_dicts]

def get_special_char_counts(list_of_dicts):
    ret = []
    for d in list_of_dicts:
        sum = 0
        for l in "-./@?&=_":
            sum += d["url"].count(l)
        ret.append(sum)
    
    return ret

def get_urls_is_https(list_of_dicts):
    return ["https" in d["url"] for d in list_of_dicts]

def get_urls_is_http(list_of_dicts):
    return ["http" in d["url"] and "https" not in d["url"] for d in list_of_dicts]

def get_known_TLD_in_domains(list_of_dicts):
    common_tlds = ["com", "org", "net", "de", "edu", "gov"]
    return [d["tld"] in common_tlds for d in list_of_dicts]



In [82]:
# build the legitimate_url_df

def build_df(list_of_dict):
    df = pd.DataFrame(\
    {"id": get_ids(list_of_dict), \
    "url": get_urls(list_of_dict), \
    "class":get_classes(list_of_dict), \
    "word_raw_count":get_word_raw_counts(list_of_dict), \
    "word_brand_count": get_word_brand_counts(list_of_dict), \
    "avg_word_raw_len": get_avg_word_raw_len(list_of_dict), \
    "longest_raw_len": get_longest_raw_lens(list_of_dict), \
    "shortest_raw_len": get_shortest_raw_lens(list_of_dict), \
    "word_composed_count": get_word_composed_counts(list_of_dict), \
    "avg_word_composed_len": get_avg_word_composed_len(list_of_dict), \
    "word_keyword_count": get_word_keyword_counts(list_of_dict), \
    "word_malicious_count": get_word_malisious_counts(list_of_dict), 
    "word_random_count": get_word_random_counts(list_of_dict), 
    "word_list_count": get_word_list_counts(list_of_dict), 
    "digit_count": get_digit_counts(list_of_dict), \
    "subdomain_len": get_subdomain_lens(list_of_dict), \
    "subdomain_count": get_subdomain_counts(list_of_dict), \
    "url_len": get_url_lens(list_of_dict),
    "www": get_word_is_in_urls(list_of_dict, "www"), \
    "com": get_word_is_in_urls(list_of_dict, "com"), \
    "special_character_count": get_special_char_counts(list_of_dict), \
    "https": get_urls_is_https(list_of_dict), \
    "http": get_urls_is_http(list_of_dict), \
    "known_TLD": get_known_TLD_in_domains(list_of_dict)})

    return df

legitimate_url_df = build_df(legit_dict_lst)
phishing_url_df = build_df(phish_dict_lst)

phishing_url_df

,id,url,class,word_raw_count,word_brand_count,avg_word_raw_len,longest_raw_len,shortest_raw_len,word_composed_count,avg_word_composed_len,...,digit_count,subdomain_len,subdomain_count,url_len,www,com,special_character_count,https,http,known_TLD
0,0,apple-iforget.com,phishing,2,1,6,7,5,0,0,...,0,0,0,17,False,True,2,False,False,True
1,1,safe-id-login.com,phishing,3,0,3,5,2,0,0,...,0,0,0,17,False,True,3,False,False,True
2,2,manage.netflix.com.usermanagement.key.1973573....,phishing,7,1,7,14,3,1,14,...,9,45,6,64,False,True,7,False,False,True
3,3,manage.netflix.com.usermanagement.key.1973574....,phishing,7,1,7,14,3,1,14,...,9,45,6,64,False,True,7,False,False,True
4,4,helpplusinfo01.com,phishing,1,0,14,14,14,0,0,...,2,0,0,18,False,True,1,False,False,True
5,5,icloudsupporto.com,phishing,1,0,14,14,14,0,0,...,0,0,0,18,False,True,1,False,False,True
6,6,fbverify.biz,phishing,1,0,8,8,8,0,0,...,0,0,0,12,False,False,1,False,False,False
7,7,paypal-my-cash.com,phishing,3,1,4,6,2,0,0,...,0,0,0,18,False,True,3,False,False,True
8,8,scotiabank-security.com,phishing,2,1,9,10,8,0,0,...,0,0,0,23,False,True,2,False,False,True
9,9,user-appleid-find.com,phishing,3,0,5,7,4,0,0,...,0,0,0,21,False,True,3,False,False,True
